<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/capfloor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for io::rate like formatting
def format_rate(r):
    return f"{r * 100:.4f}%"

# Helper for io::volatility like formatting
def format_vol(v):
    return f"{v * 100:.4f}%"

# Helper for checkAbsError
def check_abs_error(x1, x2, tolerance):
    return abs(x1 - x2) < tolerance

# Helper for typeToString
def type_to_string(capfloor_type):
    if capfloor_type == ql.CapFloor.Cap:
        return "cap"
    elif capfloor_type == ql.CapFloor.Floor:
        return "floor"
    elif capfloor_type == ql.CapFloor.Collar:
        return "collar"
    else:
        raise ValueError(f"unknown cap/floor type: {capfloor_type}")

class CommonVars:
    # common data setup
    def __init__(self):
        self.nominals = [100.0] # Changed to list to match Python bindings
        self.frequency = ql.Semiannual
        self.termStructure = ql.RelinkableYieldTermStructureHandle()
        self.index = ql.Euribor6M(self.termStructure)
        self.calendar = self.index.fixingCalendar()
        self.convention = ql.ModifiedFollowing
        # Use a fixed date for reproducibility
        self.today = ql.Date(15, ql.May, 2024) # Example fixed date
        # Important: Reset global evaluation date for each test setup if necessary
        # ql.Settings.instance().evaluationDate = self.today # Done in setUp usually
        self.settlementDays = 2
        self.fixingDays = 2
        self.settlement = self.calendar.advance(self.today, self.settlementDays, ql.Days)
        # Link the term structure AFTER settlement date is defined
        self.termStructure.linkTo(ql.FlatForward(self.settlement, 0.05,
                                                 ql.ActualActual(ql.ActualActual.ISDA)))

    # utilities
    def makeLeg(self, startDate, length_years):
        endDate = self.calendar.advance(startDate, length_years * ql.Years, self.convention)
        schedule = ql.Schedule(startDate, endDate, ql.Period(self.frequency), self.calendar,
                               self.convention, self.convention,
                               ql.DateGeneration.Forward, False)
        # Use IborLeg convenience function
        leg = ql.IborLeg([self.nominals[0]], schedule, self.index) \
                .withPaymentDayCounter(self.index.dayCounter()) \
                .withPaymentAdjustment(self.convention) \
                .withFixingDays([self.fixingDays]) # Pass fixing days as list
        return leg

    def makeEngine(self, volatility):
        vol_quote = ql.QuoteHandle(ql.SimpleQuote(volatility))
        # Ensure termStructure handle is linked before creating engine
        if not self.termStructure.empty():
             return ql.BlackCapFloorEngine(self.termStructure, vol_quote)
        else:
             raise RuntimeError("Term structure handle is not linked.")


    def makeBachelierEngine(self, volatility):
        vol_quote = ql.QuoteHandle(ql.SimpleQuote(volatility))
        if not self.termStructure.empty():
            return ql.BachelierCapFloorEngine(self.termStructure, vol_quote)
        else:
            raise RuntimeError("Term structure handle is not linked.")


    def makeCapFloor(self, capfloor_type, leg, strike, volatility, isLogNormal=True):
        strike_vector = [strike] # Caps/Floors take lists of strikes
        if capfloor_type == ql.CapFloor.Cap:
            result = ql.Cap(leg, strike_vector)
        elif capfloor_type == ql.CapFloor.Floor:
            result = ql.Floor(leg, strike_vector)
        else:
            raise ValueError("unknown cap/floor type")

        if isLogNormal:
            engine = self.makeEngine(volatility)
        else:
            engine = self.makeBachelierEngine(volatility)
        result.setPricingEngine(engine)
        return result

class CapFloorTests(unittest.TestCase):

    def setUp(self):
        self.original_eval_date = ql.Settings.instance().evaluationDate
        # CommonVars will set its own date, but we need to ensure it's used globally
        self.vars = CommonVars()
        ql.Settings.instance().evaluationDate = self.vars.today


    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.original_eval_date


    def testVega(self):
        """Testing cap/floor vega."""
        print("Testing cap/floor vega...")
        vars_ = self.vars # Use the instance created in setUp

        lengths = [1, 2, 3, 4, 5, 6, 7, 10, 15, 20, 30]
        vols = [0.01, 0.05, 0.10, 0.15, 0.20]
        strikes = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
        types = [ql.CapFloor.Cap, ql.CapFloor.Floor]

        start_date = vars_.termStructure.referenceDate()
        shift = 1e-8
        tolerance = 0.005 # Relative tolerance

        for length in lengths:
            for vol in vols:
                for strike in strikes:
                    for type_val in types:
                        leg = vars_.makeLeg(start_date, length)
                        # Ensure engines are created with the current state of vars_
                        cap_floor = vars_.makeCapFloor(type_val, leg, strike, vol)
                        shifted_cap_floor2 = vars_.makeCapFloor(type_val, leg, strike, vol + shift)
                        shifted_cap_floor1 = vars_.makeCapFloor(type_val, leg, strike, vol - shift)

                        value1 = shifted_cap_floor1.NPV()
                        value2 = shifted_cap_floor2.NPV()

                        numerical_vega = (value2 - value1) / (2 * shift) if shift != 0 else 0.0

                        # Get analytical vega
                        analytical_vega = cap_floor.result('vega')

                        if numerical_vega > 1.0e-4: # Only compare if numerical vega is significant
                            discrepancy = abs(numerical_vega - analytical_vega) / numerical_vega
                            self.assertLessEqual(discrepancy, tolerance,
                                                 msg=(f"failed to compute cap/floor vega:"
                                                      f"\n   length:      {length} years"
                                                      f"\n   volatility:  {format_vol(vol)}"
                                                      f"\n   strike:      {format_rate(strike)}"
                                                      f"\n   type:        {type_to_string(type_val)}"
                                                      f"\n   calculated:  {analytical_vega:.12f}"
                                                      f"\n   expected:    {numerical_vega:.12f}"
                                                      f"\n   discrepancy: {format_rate(discrepancy)}"
                                                      f"\n   tolerance:   {format_rate(tolerance)}"))

    def testStrikeDependency(self):
        """Testing cap/floor dependency on strike."""
        print("Testing cap/floor dependency on strike...")
        vars_ = self.vars

        lengths = [1, 2, 3, 5, 7, 10, 15, 20]
        vols = [0.01, 0.05, 0.10, 0.15, 0.20]
        strikes = [0.03, 0.04, 0.05, 0.06, 0.07] # Sorted strikes

        start_date = vars_.termStructure.referenceDate()

        for length in lengths:
            for vol in vols:
                cap_values = []
                floor_values = []
                for strike in strikes:
                    leg = vars_.makeLeg(start_date, length)
                    cap = vars_.makeCapFloor(ql.CapFloor.Cap, leg, strike, vol)
                    cap_values.append(cap.NPV())
                    floor = vars_.makeCapFloor(ql.CapFloor.Floor, leg, strike, vol)
                    floor_values.append(floor.NPV())

                # Check cap values are non-increasing
                for n in range(len(cap_values) - 1):
                    self.assertGreaterEqual(cap_values[n], cap_values[n+1] - 1e-12, # Allow for small tolerance
                                            msg=(f"NPV is increasing with the strike in a cap: \n"
                                                 f"    length:     {length} years\n"
                                                 f"    volatility: {format_vol(vol)}\n"
                                                 f"    value:      {cap_values[n]}"
                                                 f" at strike: {format_rate(strikes[n])}\n"
                                                 f"    value:      {cap_values[n+1]}"
                                                 f" at strike: {format_rate(strikes[n+1])}"))

                # Check floor values are non-decreasing
                for n in range(len(floor_values) - 1):
                    self.assertLessEqual(floor_values[n], floor_values[n+1] + 1e-12, # Allow for small tolerance
                                         msg=(f"NPV is decreasing with the strike in a floor: \n"
                                              f"    length:     {length} years\n"
                                              f"    volatility: {format_vol(vol)}\n"
                                              f"    value:      {floor_values[n]}"
                                              f" at strike: {format_rate(strikes[n])}\n"
                                              f"    value:      {floor_values[n+1]}"
                                              f" at strike: {format_rate(strikes[n+1])}"))

    def testConsistency(self):
        """Testing consistency between cap, floor and collar."""
        print("Testing consistency between cap, floor and collar...")
        vars_ = self.vars

        lengths = [1, 2, 3, 5, 7, 10, 15, 20]
        cap_rates = [0.03, 0.04, 0.05, 0.06, 0.07]
        floor_rates = [0.03, 0.04, 0.05, 0.06, 0.07]
        vols = [0.01, 0.05, 0.10, 0.15, 0.20]
        tolerance = 1.0e-10

        start_date = vars_.termStructure.referenceDate()

        for length in lengths:
            for cap_rate in cap_rates:
                for floor_rate in floor_rates:
                    for vol in vols:
                        leg = vars_.makeLeg(start_date, length)
                        cap = vars_.makeCapFloor(ql.CapFloor.Cap, leg, cap_rate, vol)
                        floor = vars_.makeCapFloor(ql.CapFloor.Floor, leg, floor_rate, vol)

                        # Collar takes lists of cap and floor rates
                        collar = ql.Collar(leg, [cap_rate], [floor_rate])
                        collar.setPricingEngine(vars_.makeEngine(vol))

                        self.assertAlmostEqual((cap.NPV() - floor.NPV()), collar.NPV(), delta=tolerance,
                                               msg=(f"inconsistency between cap, floor and collar:\n"
                                                    f"    length:       {length} years\n"
                                                    f"    volatility:   {format_vol(vol)}\n"
                                                    f"    cap value:    {cap.NPV()}"
                                                    f" at strike: {format_rate(cap_rate)}\n"
                                                    f"    floor value:  {floor.NPV()}"
                                                    f" at strike: {format_rate(floor_rate)}\n"
                                                    f"    collar value: {collar.NPV()}"))

                        # Test optionlet recomposition
                        num_optionlets = len(leg) # Assuming one optionlet per leg period

                        # Caplets
                        caplets_npv = 0.0
                        for m in range(num_optionlets):
                             optionlet = cap.optionlet(m)
                             optionlet.setPricingEngine(vars_.makeEngine(vol))
                             caplets_npv += optionlet.NPV()
                        self.assertAlmostEqual(cap.NPV(), caplets_npv, delta=tolerance,
                                               msg=(f"sum of caplet NPVs does not equal cap NPV:\n"
                                                    f"    length: {length} years, vol: {format_vol(vol)}, strike: {format_rate(cap_rate)}\n"
                                                    f"    cap NPV: {cap.NPV()}, sum caplets: {caplets_npv}"))

                        # Floorlets
                        floorlets_npv = 0.0
                        for m in range(num_optionlets):
                             optionlet = floor.optionlet(m)
                             optionlet.setPricingEngine(vars_.makeEngine(vol))
                             floorlets_npv += optionlet.NPV()
                        self.assertAlmostEqual(floor.NPV(), floorlets_npv, delta=tolerance,
                                               msg=(f"sum of floorlet NPVs does not equal floor NPV:\n"
                                                    f"    length: {length} years, vol: {format_vol(vol)}, strike: {format_rate(floor_rate)}\n"
                                                    f"    floor NPV: {floor.NPV()}, sum floorlets: {floorlets_npv}"))

                        # Collarlets
                        collarlets_npv = 0.0
                        for m in range(num_optionlets):
                             optionlet = collar.optionlet(m)
                             optionlet.setPricingEngine(vars_.makeEngine(vol))
                             collarlets_npv += optionlet.NPV()
                        self.assertAlmostEqual(collar.NPV(), collarlets_npv, delta=tolerance,
                                               msg=(f"sum of collarlet NPVs does not equal collar NPV:\n"
                                                    f"    length: {length} years, vol: {format_vol(vol)}, cap_k: {format_rate(cap_rate)}, floor_k: {format_rate(floor_rate)}\n"
                                                    f"    collar NPV: {collar.NPV()}, sum collarlets: {collarlets_npv}"))


    def testParity(self):
        """Testing cap/floor parity."""
        print("Testing cap/floor parity...")
        vars_ = self.vars

        lengths = [1, 2, 3, 5, 7, 10, 15, 20]
        strikes = [0.0, 0.03, 0.04, 0.05, 0.06, 0.07]
        vols = [0.01, 0.05, 0.10, 0.15, 0.20]
        tolerance = 1.0e-10

        start_date = vars_.termStructure.referenceDate()

        for length in lengths:
            for strike in strikes:
                for vol in vols:
                    leg = vars_.makeLeg(start_date, length)
                    cap = vars_.makeCapFloor(ql.CapFloor.Cap, leg, strike, vol)
                    floor = vars_.makeCapFloor(ql.CapFloor.Floor, leg, strike, vol)

                    # Create the corresponding swap
                    maturity = vars_.calendar.advance(start_date, length, ql.Years, vars_.convention)
                    # Need separate schedules for fixed and float legs if details differ
                    # Here, they seem to share the same schedule parameters
                    schedule = ql.Schedule(start_date, maturity, ql.Period(vars_.frequency),
                                           vars_.calendar, vars_.convention, vars_.convention,
                                           ql.DateGeneration.Forward, False)

                    swap = ql.VanillaSwap(ql.Swap.Payer, vars_.nominals[0],
                                          schedule, strike, vars_.index.dayCounter(), # Fixed leg details
                                          schedule, vars_.index, 0.0, vars_.index.dayCounter()) # Float leg details
                    swap.setPricingEngine(ql.DiscountingSwapEngine(vars_.termStructure))

                    self.assertAlmostEqual(cap.NPV() - floor.NPV(), swap.NPV(), delta=tolerance,
                                           msg=(f"put/call parity violated:\n"
                                                f"    length:      {length} years\n"
                                                f"    volatility:  {format_vol(vol)}\n"
                                                f"    strike:      {format_rate(strike)}\n"
                                                f"    cap value:   {cap.NPV()}\n"
                                                f"    floor value: {floor.NPV()}\n"
                                                f"    swap value:  {swap.NPV()}"))

    def testATMRate(self):
        """Testing cap/floor ATM rate."""
        print("Testing cap/floor ATM rate...")
        vars_ = self.vars

        lengths = [1, 2, 3, 5, 7, 10, 15, 20]
        # Strikes don't affect ATM rate, but used in loop
        strikes = [0.0, 0.03, 0.04, 0.05, 0.06, 0.07]
        vols = [0.01, 0.05, 0.10, 0.15, 0.20]
        tolerance = 1.0e-10

        start_date = vars_.termStructure.referenceDate()

        for length in lengths:
            leg = vars_.makeLeg(start_date, length)
            maturity = vars_.calendar.advance(start_date, length, ql.Years, vars_.convention)
            schedule = ql.Schedule(start_date, maturity, ql.Period(vars_.frequency),
                                   vars_.calendar, vars_.convention, vars_.convention,
                                   ql.DateGeneration.Forward, False)

            for strike in strikes: # Strike used to create cap/floor, but ATM rate is independent
                for vol in vols:
                    cap = vars_.makeCapFloor(ql.CapFloor.Cap, leg, strike, vol)
                    floor = vars_.makeCapFloor(ql.CapFloor.Floor, leg, strike, vol)

                    # Pass the dereferenced term structure to atmRate
                    cap_atm_rate = cap.atmRate(vars_.termStructure.currentLink())
                    floor_atm_rate = floor.atmRate(vars_.termStructure.currentLink())

                    self.assertTrue(check_abs_error(floor_atm_rate, cap_atm_rate, tolerance),
                                    msg=(f"Cap ATM Rate and floor ATM Rate should be equal:\n"
                                         f"   length:        {length} years\n"
                                         f"   volatility:    {format_vol(vol)}\n"
                                         f"   (irrelevant) strike: {format_rate(strike)}\n"
                                         f"   cap ATM rate:  {cap_atm_rate}\n"
                                         f"   floor ATM rate:{floor_atm_rate}\n"
                                         f"   abs diff:      {abs(cap_atm_rate - floor_atm_rate)}"))

                    # Check swap NPV at ATM rate
                    atm_swap = ql.VanillaSwap(ql.Swap.Payer, vars_.nominals[0],
                                              schedule, floor_atm_rate, vars_.index.dayCounter(),
                                              schedule, vars_.index, 0.0, vars_.index.dayCounter())
                    atm_swap.setPricingEngine(ql.DiscountingSwapEngine(vars_.termStructure))
                    swap_npv = atm_swap.NPV()

                    self.assertTrue(check_abs_error(swap_npv, 0.0, tolerance),
                                    msg=(f"the NPV of a Swap struck at ATM rate should be zero:\n"
                                         f"   length:        {length} years\n"
                                         f"   volatility:    {format_vol(vol)}\n"
                                         f"   ATM rate:      {format_rate(floor_atm_rate)}\n"
                                         f"   swap NPV:      {swap_npv}"))


    def testImpliedVolatility(self):
        """Testing implied term volatility for cap and floor."""
        print("Testing implied term volatility for cap and floor...")
        vars_ = self.vars

        max_evaluations = 100
        tolerance = 1.0e-8 # Tolerance for implied vol recovery

        types = [ql.CapFloor.Cap, ql.CapFloor.Floor]
        strikes = [0.02, 0.03, 0.04]
        lengths = [1, 5, 10]
        r_rates = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
        vols = [0.01, 0.05, 0.10, 0.20, 0.30, 0.70, 0.90]

        for length in lengths:
            leg = vars_.makeLeg(vars_.settlement, length)
            for type_val in types:
                for strike in strikes:
                    # Create cap/floor with zero vol initially to get structure
                    capfloor = vars_.makeCapFloor(type_val, leg, strike, 0.0)

                    for r in r_rates:
                        # Update term structure
                        vars_.termStructure.linkTo(ql.FlatForward(vars_.settlement, r, ql.Actual360()))

                        for v in vols:
                            capfloor.setPricingEngine(vars_.makeEngine(v))
                            value = capfloor.NPV()
                            impl_vol = 0.0
                            impl_vol_exception = None

                            try:
                                impl_vol = capfloor.impliedVolatility(value,
                                                                      vars_.termStructure,
                                                                      0.10, # guess
                                                                      tolerance, # accuracy
                                                                      max_evaluations,
                                                                      1.0e-7, # min vol
                                                                      4.0,    # max vol
                                                                      ql.ShiftedLognormal, # vol type
                                                                      0.0) # displacement
                            except Exception as e:
                                impl_vol_exception = e
                                # Check if price is very close to price at vol=0
                                capfloor.setPricingEngine(vars_.makeEngine(0.0))
                                value2 = capfloor.NPV()
                                if abs(value - value2) < tolerance:
                                    # Price is essentially intrinsic value, implied vol is ~0. Skip failure.
                                    continue
                                else:
                                    # Real failure
                                    self.fail(f"implied vol failure (exception): {type_to_string(type_val)}\n"
                                              f"  strike: {format_rate(strike)}, rate: {format_rate(r)}, len: {length}Y\n"
                                              f"  vol: {format_vol(v)}, price: {value}\n Error: {e}")

                            # Check recovered vol
                            if abs(impl_vol - v) > tolerance:
                                # Re-price with implied vol and check price consistency
                                capfloor.setPricingEngine(vars_.makeEngine(impl_vol))
                                value2 = capfloor.NPV()
                                if abs(value - value2) > tolerance:
                                    self.fail(f"implied vol failure (price mismatch): {type_to_string(type_val)}\n"
                                              f"  strike: {format_rate(strike)}, rate: {format_rate(r)}, len: {length}Y\n"
                                              f"  vol: {format_vol(v)}, price: {value}\n"
                                              f"  implied vol: {format_vol(impl_vol)}, implied price: {value2}")


    def testCachedValue(self):
        """Testing Black cap/floor price against cached values."""
        print("Testing Black cap/floor price against cached values...")
        vars_ = self.vars # Use instance setup
        original_eval_date = ql.Settings.instance().evaluationDate

        cached_today = ql.Date(14, ql.March, 2002)
        cached_settlement = ql.Date(18, ql.March, 2002)
        ql.Settings.instance().evaluationDate = cached_today
        # Need to update settlement in vars_ or create new TS
        vars_.settlement = cached_settlement
        vars_.termStructure.linkTo(ql.FlatForward(cached_settlement, 0.05, ql.Actual360()))

        start_date = vars_.termStructure.referenceDate() # Should be cachedSettlement now
        leg = vars_.makeLeg(start_date, 20)

        cap = vars_.makeCapFloor(ql.CapFloor.Cap, leg, 0.07, 0.20)
        floor = vars_.makeCapFloor(ql.CapFloor.Floor, leg, 0.03, 0.20)

        # Adjust expected based on par coupon setting
        using_par = ql.IborCoupon.Settings.instance().usingAtParCoupons()
        cached_cap_npv   = 6.87570026732 if using_par else 6.87630307745
        cached_floor_npv = 2.65812927959 if using_par else 2.65796764715

        self.assertAlmostEqual(cap.NPV(), cached_cap_npv, delta=1.0e-11,
                               msg=(f"failed to reproduce cached cap value:\n"
                                    f"    calculated: {cap.NPV():.12f}\n"
                                    f"    expected:   {cached_cap_npv:.12f}"))
        self.assertAlmostEqual(floor.NPV(), cached_floor_npv, delta=1.0e-11,
                               msg=(f"failed to reproduce cached floor value:\n"
                                    f"    calculated: {floor.NPV():.12f}\n"
                                    f"    expected:   {cached_floor_npv:.12f}"))

        ql.Settings.instance().evaluationDate = original_eval_date


    def testCachedValueFromOptionLets(self):
        """Testing Black cap/floor price as a sum of optionlets prices against cached values."""
        print("Testing Black cap/floor price as a sum of optionlets prices against cached values...")
        vars_ = self.vars
        original_eval_date = ql.Settings.instance().evaluationDate

        cached_today = ql.Date(14, ql.March, 2002)
        cached_settlement = ql.Date(18, ql.March, 2002)
        ql.Settings.instance().evaluationDate = cached_today
        vars_.settlement = cached_settlement
        base_curve = ql.FlatForward(cached_settlement, 0.05, ql.Actual360())
        vars_.termStructure.linkTo(base_curve)

        start_date = vars_.termStructure.referenceDate()
        leg = vars_.makeLeg(start_date, 20)

        cap = vars_.makeCapFloor(ql.CapFloor.Cap, leg, 0.07, 0.20)
        floor = vars_.makeCapFloor(ql.CapFloor.Floor, leg, 0.03, 0.20)

        using_par = ql.IborCoupon.Settings.instance().usingAtParCoupons()
        cached_cap_npv   = 6.87570026732 if using_par else 6.87630307745
        cached_floor_npv = 2.65812927959 if using_par else 2.65796764715

        # Get optionlet prices using result()
        caplet_prices = cap.result('optionletsPrice')
        floorlet_prices = floor.result('optionletsPrice')

        self.assertEqual(len(caplet_prices), 40, # 20 years, Semiannual = 40 periods
                         msg=f"Expected 40 caplet prices, got {len(caplet_prices)}")

        calculated_caplets_npv = sum(caplet_prices)
        calculated_floorlets_npv = sum(floorlet_prices)

        self.assertAlmostEqual(calculated_caplets_npv, cached_cap_npv, delta=1.0e-11,
                               msg=(f"failed to reproduce cached cap value from its caplets' values:\n"
                                    f"    calculated: {calculated_caplets_npv:.12f}\n"
                                    f"    expected:   {cached_cap_npv:.12f}"))
        self.assertAlmostEqual(calculated_floorlets_npv, cached_floor_npv, delta=1.0e-11,
                               msg=(f"failed to reproduce cached floor value from its floorlets' values:\n"
                                    f"    calculated: {calculated_floorlets_npv:.12f}\n"
                                    f"    expected:   {cached_floor_npv:.12f}"))

        ql.Settings.instance().evaluationDate = original_eval_date

    def _run_optionlets_delta_test(self, is_bachelier):
        """Helper for testing optionlet deltas (Black or Bachelier)."""
        model_name = "Bachelier" if is_bachelier else "Black"
        print(f"Testing {model_name} caplet/floorlet delta coefficients against finite difference values...")
        vars_ = self.vars
        original_eval_date = ql.Settings.instance().evaluationDate

        cached_today = ql.Date(14, ql.March, 2002)
        cached_settlement = ql.Date(18, ql.March, 2002)
        ql.Settings.instance().evaluationDate = cached_today
        vars_.settlement = cached_settlement

        base_curve = ql.FlatForward(cached_settlement, 0.05, ql.Actual360())
        base_curve_handle = ql.RelinkableYieldTermStructureHandle(base_curve)

        eps = 1.0e-6
        spread_quote = ql.SimpleQuote(0.0)
        spread_handle = ql.QuoteHandle(spread_quote)
        spread_curve = ql.ZeroSpreadedTermStructure(base_curve_handle, spread_handle,
                                                   ql.Continuous, ql.Annual, ql.Actual360())
        vars_.termStructure.linkTo(spread_curve) # Link main handle to spreaded curve

        start_date = vars_.termStructure.referenceDate()
        leg = vars_.makeLeg(start_date, 20)

        vol = 0.20 if not is_bachelier else 0.01 # Use appropriate vol for model
        strike = 0.05
        cap = vars_.makeCapFloor(ql.CapFloor.Cap, leg, strike, vol, isLogNormal=not is_bachelier)
        floor = vars_.makeCapFloor(ql.CapFloor.Floor, leg, strike, vol, isLogNormal=not is_bachelier)

        # Get analytical deltas first (with spread=0)
        caplet_analytic_delta = cap.result('optionletsDelta')
        floorlet_analytic_delta = floor.result('optionletsDelta')

        # Finite difference calculation
        spread_quote.setValue(eps)
        caplet_up_prices = cap.result('optionletsPrice')
        floorlet_up_prices = floor.result('optionletsPrice')
        caplet_df_up = cap.result('optionletsDiscountFactor')
        floorlet_df_up = floor.result('optionletsDiscountFactor')
        caplet_fwd_up = cap.result('optionletsAtmForward')
        floorlet_fwd_up = floor.result('optionletsAtmForward')

        spread_quote.setValue(-eps)
        caplet_down_prices = cap.result('optionletsPrice')
        floorlet_down_prices = floor.result('optionletsPrice')
        caplet_df_down = cap.result('optionletsDiscountFactor')
        floorlet_df_down = floor.result('optionletsDiscountFactor')
        caplet_fwd_down = cap.result('optionletsAtmForward')
        floorlet_fwd_down = floor.result('optionletsAtmForward')

        # Reset spread
        spread_quote.setValue(0.0)

        # Calculate FD deltas
        cap_leg = cap.floatingLeg()
        floor_leg = floor.floatingLeg()
        num_caplets = len(cap_leg)
        num_floorlets = len(floor_leg)

        caplet_fd_delta = [0.0] * num_caplets
        for n in range(num_caplets):
            # Cast to FloatingRateCoupon to access details
            # Note: Need to handle potential errors if cast fails or index out of bounds
            coupon = ql.as_floating_rate_coupon(cap_leg[n])
            if coupon:
                accrual_factor = coupon.nominal() * coupon.accrualPeriod() * coupon.gearing()
                if abs(accrual_factor) > 1e-16 and abs(caplet_fwd_up[n] - caplet_fwd_down[n]) > 1e-16:
                    # Adjusted price difference / forward rate difference / accrual factor
                    price_diff_adj = (caplet_up_prices[n] / caplet_df_up[n] -
                                      caplet_down_prices[n] / caplet_df_down[n])
                    fwd_diff = caplet_fwd_up[n] - caplet_fwd_down[n]
                    caplet_fd_delta[n] = price_diff_adj / fwd_diff / accrual_factor
                else:
                     caplet_fd_delta[n] = 0.0 # Avoid division by zero

        floorlet_fd_delta = [0.0] * num_floorlets
        for n in range(num_floorlets):
            coupon = ql.as_floating_rate_coupon(floor_leg[n])
            if coupon:
                accrual_factor = coupon.nominal() * coupon.accrualPeriod() * coupon.gearing()
                if abs(accrual_factor) > 1e-16 and abs(floorlet_fwd_up[n] - floorlet_fwd_down[n]) > 1e-16:
                    price_diff_adj = (floorlet_up_prices[n] / floorlet_df_up[n] -
                                      floorlet_down_prices[n] / floorlet_df_down[n])
                    fwd_diff = floorlet_fwd_up[n] - floorlet_fwd_down[n]
                    floorlet_fd_delta[n] = price_diff_adj / fwd_diff / accrual_factor
                else:
                    floorlet_fd_delta[n] = 0.0

        # Compare Analytical vs FD
        tolerance_delta = 1.0e-6
        # Caplets: C++ test starts loop from n=1. Why? First fixing? Let's check all.
        # If first coupon's forward is fixed, its delta might be zero or handled differently.
        # Let's compare all and see. Python indices start at 0.
        for n in range(num_caplets):
             # Skip comparison if analytical delta is very small (potential noise in FD)
             if abs(caplet_analytic_delta[n]) > 1e-9:
                  self.assertAlmostEqual(caplet_fd_delta[n], caplet_analytic_delta[n], delta=tolerance_delta,
                                         msg=(f"failed to compare {model_name} caplet delta (Analytic vs FD):\n"
                                              f"caplet number: {n}\n"
                                              f"    finite difference: {caplet_fd_delta[n]:.12f}\n"
                                              f"    analytical value:  {caplet_analytic_delta[n]:.12f}"))

        for n in range(num_floorlets):
            if abs(floorlet_analytic_delta[n]) > 1e-9:
                  self.assertAlmostEqual(floorlet_fd_delta[n], floorlet_analytic_delta[n], delta=tolerance_delta,
                                         msg=(f"failed to compare {model_name} floorlet delta (Analytic vs FD):\n"
                                              f"floorlet number: {n}\n"
                                              f"    finite difference: {floorlet_fd_delta[n]:.12f}\n"
                                              f"    analytical value:  {floorlet_analytic_delta[n]:.12f}"))

        ql.Settings.instance().evaluationDate = original_eval_date

    def testOptionLetsDelta(self):
        """Wrapper for Black model optionlet delta test."""
        self._run_optionlets_delta_test(is_bachelier=False)

    def testBachelierOptionLetsDelta(self):
        """Wrapper for Bachelier model optionlet delta test."""
        self._run_optionlets_delta_test(is_bachelier=True)

if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)